In [1]:
import json

# --- PLEASE EDIT THIS VARIABLE ---
json_file_path = r"D:\project backup\ROSProject\RosProject\pose_estimation\UR3_single_cube_training\captures_000.json"
# --------------------------------

try:
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    print(f"✅ Successfully loaded '{json_file_path}'")
    
    captures = data.get('captures', [])
    if not captures:
        print("🔴 ERROR: The 'captures' list is empty.")
    else:
        # Check the first annotation in the file
        first_capture = captures[0]
        annotations = first_capture.get('annotations', [{}])[0]
        values = annotations.get('values', [])

        print(f"\n--- Analysis of First Capture ---")
        if not values:
            print("🔴 ERROR: The 'values' list for the first capture is EMPTY.")
            print("This is the reason no pose data is being loaded.")
        else:
            print("🟢 SUCCESS: Found annotation data in the 'values' list.")
            print("Sample annotation:", values[0])

except FileNotFoundError:
    print(f"❌ ERROR: File not found. Please double-check the path: {json_file_path}")
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Successfully loaded 'D:\project backup\ROSProject\RosProject\pose_estimation\UR3_single_cube_training\captures_000.json'

--- Analysis of First Capture ---
🟢 SUCCESS: Found annotation data in the 'values' list.
Sample annotation: {'label_id': 0, 'label_name': 'cube_position', 'instance_id': 1, 'translation': {'x': -0.2266370952129364, 'y': -0.06708422303199768, 'z': 4.477364540100098}, 'size': {'x': 0.15311414003372192, 'y': 0.11453963071107864, 'z': 0.11453963071107864}, 'rotation': {'x': -0.2964995503425598, 'y': 0.5840931534767151, 'z': -0.24193927645683289, 'w': 0.7158132791519165}, 'velocity': {'x': 0.0, 'y': 0.0, 'z': 0.0}, 'acceleration': {'x': 0.0, 'y': 0.0, 'z': 0.0}}


In [7]:
import os
import sys
import yaml
import json
from easydict import EasyDict
from PIL import Image

# --- Setup and Configuration ---
# Add the project root to the Python path to allow imports
project_root = os.getcwd()
if project_root not in sys.path:
    # Handle the case where the project name might be duplicated in the path
    if os.path.basename(project_root) == "RosProject":
         parent_dir = os.path.dirname(project_root)
         if os.path.basename(parent_dir) == "RosProject":
             project_root = parent_dir
    
    if project_root not in sys.path:
        sys.path.insert(0, project_root)

print(f"Project root identified as: {project_root}")

# Define paths.
config_file = 'config.yaml'
# This path should point to the directory that CONTAINS the training data folder.
data_root = os.path.join(project_root, 'pose_estimation')


# --- Direct Inspection Logic ---

def direct_data_inspection(config_path, data_root_path):
    """
    Manually inspects data files and paths without using the custom iterator class.
    """
    print("\n--- Starting Direct Data Inspection ---")

    # 1. Load configuration
    try:
        with open(config_path, 'r') as f:
            config = EasyDict(yaml.safe_load(f))
        print(f"✅ Configuration loaded from '{config_path}'")
    except FileNotFoundError:
        print(f"❌ ERROR: Config file not found at '{config_path}'")
        return

    # 2. Define key paths
    train_folder_name = config.train.dataset_zip_file_name_training
    base_path = os.path.join(data_root_path, train_folder_name)
    
    print(f"\n[Step 1] Checking for the main training data folder...")
    print(f"         -> Path: {base_path}")
    if not os.path.isdir(base_path):
        print(f"❌ ERROR: Main training folder not found at the path above.")
        return
    print(f"✅ Found main training folder.")

    # 3. Find the 'Dataset' subfolder
    dataset_folder_path = None
    for file in os.listdir(base_path):
        if file.startswith("Dataset") and os.path.isdir(os.path.join(base_path, file)):
            dataset_folder_path = os.path.join(base_path, file)
            break
    
    print(f"\n[Step 2] Looking for 'Dataset' subfolder inside training folder...")
    if not dataset_folder_path:
        print(f"❌ ERROR: Could not find a subfolder starting with 'Dataset' inside {base_path}")
        return
    print(f"✅ Found 'Dataset' subfolder at: {dataset_folder_path}")

    # 4. Find and inspect the first JSON annotation file
    json_path = os.path.join(dataset_folder_path, 'captures_000.json')
    print(f"\n[Step 3] Checking for the first annotation file...")
    print(f"         -> Path: {json_path}")
    if not os.path.isfile(json_path):
        print(f"❌ ERROR: Annotation file 'captures_000.json' not found.")
        return
    print(f"✅ Found annotation file.")

    # 5. Load JSON and inspect the first capture
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        capture = data['captures'][0]
        image_relative_path = capture['filename']
        print(f"\n[Step 4] Reading first capture from JSON...")
        print(f"✅ Annotation data seems valid.")
        print(f"         -> Image relative path from JSON: '{image_relative_path}'")
        
        # Check for non-empty annotations
        values = capture['annotations'][0]['values']
        if not values:
             print("❌ ERROR: The 'values' array in this capture is empty. No pose data available.")
             return
        print("✅ Annotations 'values' array is not empty.")

    except (json.JSONDecodeError, IndexError, KeyError) as e:
        print(f"❌ ERROR: Failed to parse '{json_path}' or find required data fields. Error: {e}")
        return

    # 6. Construct the final image path and verify it
    final_image_path = os.path.join(base_path, image_relative_path)
    print(f"\n[Step 5] Constructing and verifying final image path...")
    print(f"         -> Base Path: '{base_path}'")
    print(f"         -> Relative Path: '{image_relative_path}'")
    print(f"         -> FINAL PATH: '{final_image_path}'")

    try:
        with Image.open(final_image_path) as img:
            print(f"\n🎉 SUCCESS! Image file found and opened successfully.")
            print(f"   Image format: {img.format}, Size: {img.size}")
            print("\nThis confirms your data paths and structure are correct. The original issue might be related to other annotation files being empty or a subtle bug in the original data loader.")
            
    except FileNotFoundError:
        print(f"\n🔴 FINAL ERROR: File not found at the constructed path.")
        print("   This is the root cause of the problem. The path constructed from your base folder and the filename from the JSON is incorrect.")
        print("   Please check for typos, extra folders, or incorrect path separators.")
    except Exception as e:
        print(f"\n🔴 FINAL ERROR: File found, but could not be opened. It might be corrupt.")
        print(f"   Error: {e}")


# --- Run the Inspection ---
direct_data_inspection(config_file, data_root)



Project root identified as: D:\project backup\ROSproject\RosProject

--- Starting Direct Data Inspection ---
✅ Configuration loaded from 'config.yaml'

[Step 1] Checking for the main training data folder...
         -> Path: D:\project backup\ROSproject\RosProject\pose_estimation\UR3_single_cube_training
✅ Found main training folder.

[Step 2] Looking for 'Dataset' subfolder inside training folder...
✅ Found 'Dataset' subfolder at: D:\project backup\ROSproject\RosProject\pose_estimation\UR3_single_cube_training\Datasetdc3e8deb-8995-42f4-8f21-277f73fdedb9

[Step 3] Checking for the first annotation file...
         -> Path: D:\project backup\ROSproject\RosProject\pose_estimation\UR3_single_cube_training\Datasetdc3e8deb-8995-42f4-8f21-277f73fdedb9\captures_000.json
❌ ERROR: Annotation file 'captures_000.json' not found.
